In [13]:
import pandas as pd
import numpy as np
import sagemaker
from sagemaker.predictor import csv_serializer
import boto3
import re
import os
from sagemaker import get_execution_role

role = get_execution_role()

df=pd.read_csv('s3://sage-maker-3/DadosTeseLogit.csv',sep=',',header=0)
sel=np.where(df.corr()['selected']>.5)[0][0:-1]
df=df.iloc[:,np.concatenate([[30],sel])]

In [14]:
containers  = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest',
              'ap-northeast-1': '501404015308.dkr.ecr.ap-northeast-1.amazonaws.com/xgboost:latest'}

In [15]:
bucket =  'sage-maker-3'
prefix = 'sagemaker/xgboost-churn'

In [16]:
df.to_csv('train.csv', header=False, index=False)
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')

In [17]:

sess =sagemaker.Session()

xgb = sagemaker.estimator.Estimator(containers[boto3.Session().region_name],
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=20,
                        min_child_weight=0.1,
                        subsample=0.95,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

xgb.fit({'train': s3_input_train})


INFO:sagemaker:Creating training-job with name: xgboost-2018-06-01-23-55-31-429


...................
Arguments: train
[2018-06-01:23:58:35:INFO] Running standalone xgboost training.
[2018-06-01:23:58:35:INFO] Path /opt/ml/input/data/validation does not exist!
[2018-06-01:23:58:35:INFO] File size need to be processed in the node: 0.0mb. Available memory size in the node: 8616.58mb
[2018-06-01:23:58:35:INFO] Deduced delimiter of CSV input is ','
[23:58:35] S3DistributionType set as FullyReplicated
[23:58:35] 98x3 matrix with 294 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[23:58:35] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[0]#011train-error:0
[23:58:35] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[1]#011train-error:0
[23:58:35] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[2]#011train-error:0
[23:58:35] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2

===== Job Complete =====
Billable seconds: 90


In [18]:
xgb_predictor = xgb.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: xgboost-2018-06-02-00-00-29-369
INFO:sagemaker:Creating endpoint with name xgboost-2018-06-01-23-55-31-429


---------------------------------------------------------------!

In [21]:
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer
xgb_predictor.deserializer = None


In [25]:
df3=df.iloc[:,[1,2,3]]

predictions=xgb_predictor.predict(df3.as_matrix()).decode('utf-8')